In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import mmfreelm
from transformers import AutoModelForCausalLM, AutoTokenizer
#Change here to our open-sourced model
# name = 'ridger/MMfreeLM-1.3B'
name = 'ridger/MMfreeLM-370M'
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name).cuda().half()
# input_prompt = "In a shocking finding, scientist discovered a herd of unicorns living in a remote, "
input_prompt = "What is 1 + 1? \nA: 1 + 1 ="
input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.cuda()
# outputs = model.generate(input_ids, max_length=36,  do_sample=True, top_p=0.4, temperature=0.6)
outputs = model.generate(input_ids, max_length=36,  do_sample=False)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is 1 + 1? 
A: 1 + 1 = 1 + 1 + 1 + 1 + 1 + 1


In [2]:
model.model(input_ids)

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.2188,  0.4482,  0.2458,  ..., -0.3171,  0.2125, -0.3545],
         [-0.0437, -0.4780, -0.1870,  ..., -0.2944,  0.3066,  0.2449],
         [ 0.2490,  0.0030,  0.1798,  ...,  0.0019, -0.0635,  0.5825],
         ...,
         [-0.0367,  0.1432,  0.2932,  ...,  0.1146, -0.2417,  0.0828],
         [-0.2316,  0.4299,  0.1481,  ..., -0.0165,  0.0116,  0.4966],
         [ 0.1337,  0.7725,  0.6509,  ...,  0.0709, -0.0008,  0.2139]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<LayerNormFnBackward>), past_key_values=((tensor([[[25.0156, 15.3594, -0.1969,  ..., 19.5000, 21.9219, 43.5625]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<CopyBackwards>),), (tensor([[[-0.2257, -0.1931, -0.2737,  ..., -0.2627, -0.2690, -0.1904]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<CopyBackwards>),), (tensor([[[-0.2454, -0.2458, -0.1927,  ..., -0.2196,  0.2373, -0.1458]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<

In [3]:
o1 = model.lm_head(model.model(input_ids).last_hidden_state)
o1.shape

torch.Size([1, 19, 32000])

In [4]:
tokenizer.decode(input_ids[0])

'<s> What is 1 + 1? \nA: 1 + 1 ='

In [5]:
tokenizer.decode(o1[0].argmax(-1)[-1])

''

In [6]:
model.model.embeddings.weight.shape

torch.Size([32000, 1024])

In [7]:
model.model

HGRNBitModel(
  (embeddings): Embedding(32000, 1024)
  (layers): ModuleList(
    (0-23): 24 x HGRNBitBlock(
      (attn_norm): RMSNorm(1024, eps=1e-06)
      (attn): HGRNBitAttention(
        (i_proj): FusedBitLinear(
          in_features=1024, out_features=1024, bias=False
          (norm): RMSNorm(1024, eps=1e-08)
        )
        (f_proj): FusedBitLinear(
          in_features=1024, out_features=1024, bias=False
          (norm): RMSNorm(1024, eps=1e-08)
        )
        (g_proj): FusedBitLinear(
          in_features=1024, out_features=1024, bias=False
          (norm): RMSNorm(1024, eps=1e-08)
        )
        (g_norm): FusedRMSNormSwishGate()
        (o_proj): FusedBitLinear(
          in_features=1024, out_features=1024, bias=False
          (norm): RMSNorm(1024, eps=1e-08)
        )
      )
      (mlp_norm): RMSNorm(1024, eps=1e-06)
      (mlp): HGRNBitMLP(
        (gate_proj): FusedBitLinear(
          in_features=1024, out_features=5632, bias=False
          (norm): RMSNo

In [8]:
from igre_v1 import IEncoderPretrainedHGRN

encoder = IEncoderPretrainedHGRN(model.model)
encoder

IEncoderPretrainedHGRN(
  (hgrn_model): HGRNBitModel(
    (embeddings): Embedding(32000, 1024)
    (layers): ModuleList(
      (0-23): 24 x HGRNBitBlock(
        (attn_norm): RMSNorm(1024, eps=1e-06)
        (attn): HGRNBitAttention(
          (i_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
          (f_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
          (g_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
          (g_norm): FusedRMSNormSwishGate()
          (o_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
        )
        (mlp_norm): RMSNorm(1024, eps=1e-06)
        (mlp): HGRNBitMLP(
          (gate_proj): Fused

In [9]:
encoder(input_ids)

tensor([[[-0.2188,  0.4482,  0.2458,  ..., -0.3171,  0.2125, -0.3545],
         [-0.0437, -0.4780, -0.1870,  ..., -0.2944,  0.3066,  0.2449],
         [ 0.2490,  0.0030,  0.1798,  ...,  0.0019, -0.0635,  0.5825],
         ...,
         [-0.0367,  0.1432,  0.2932,  ...,  0.1146, -0.2417,  0.0828],
         [-0.2316,  0.4299,  0.1481,  ..., -0.0165,  0.0116,  0.4966],
         [ 0.1337,  0.7725,  0.6509,  ...,  0.0709, -0.0008,  0.2139]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<LayerNormFnBackward>)

In [10]:
from igre_v1 import iDecoderPretrainedHGRN

decoder = iDecoderPretrainedHGRN(model.model, model.lm_head)
decoder

iDecoderPretrainedHGRN(
  (hgrn_model): HGRNBitModel(
    (embeddings): Embedding(32000, 1024)
    (layers): ModuleList(
      (0-23): 24 x HGRNBitBlock(
        (attn_norm): RMSNorm(1024, eps=1e-06)
        (attn): HGRNBitAttention(
          (i_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
          (f_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
          (g_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
          (g_norm): FusedRMSNormSwishGate()
          (o_proj): FusedBitLinear(
            in_features=1024, out_features=1024, bias=False
            (norm): RMSNorm(1024, eps=1e-08)
          )
        )
        (mlp_norm): RMSNorm(1024, eps=1e-06)
        (mlp): HGRNBitMLP(
          (gate_proj): Fused